In [33]:
import re
import selenium
from unittest import result
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,WebDriverException,StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import random
import time
from selenium.webdriver.support import expected_conditions as EC
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [72]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--headless")
chrome_driver = webdriver.Chrome(options=chrome_options)
page_url = 'https://www.parashop.tn/solaire?fc=345'
chrome_driver.get(page_url)
chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.8);")

# parashop.tn

In [69]:
def scrollButtom(driver, max_attempts=5):
    last_height = driver.execute_script("return document.body.scrollHeight")
    attempts = 0
    
    while attempts < max_attempts:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Check new height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            attempts += 1
        else:
            attempts = 0  # Reset if new content loaded
            
        last_height = new_height
    
    print("Reached the bottom!")
#scrollButtom(chrome_driver)

In [133]:
def collectProducts():
    i=49

    while True:
        try:
            scrollButtom(chrome_driver)
            path='/html/body/div[4]/div[2]/div/div/div/div[2]/div['+str(i)+']/a'
            print(path)
            click_plus= WebDriverWait(chrome_driver,5).until(EC.element_to_be_clickable((By.XPATH,path))).click()
            i=i+12
            print('clicked')
        except:
            print(i)
            try:
                time.sleep(2)
                chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.913);")
                path='/html/body/div[4]/div[2]/div/div/div/div[2]/div['+str(i)+']/a'
                print(path)
                click_plus= WebDriverWait(chrome_driver,5).until(EC.element_to_be_clickable((By.XPATH,path))).click()
                i=i+12
                print('clicked**')
                scrollButtom(chrome_driver)
            except:
                print(i, "FINAL")
                break

#collectProducts()  
    

In [148]:
def getFirstElements():
    products = chrome_driver.find_elements(By.CSS_SELECTOR, 'div.main-products.product-grid > div')
    product_data = []
    
    # Date as simple string
    scraping_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    #for index, product_element in enumerate(products, 1):
    for product_element in products:
        try:
            link = product_element.find_element(By.CSS_SELECTOR, 'div.caption div.name a')
            name= link.text.strip().lower()

            if any(word in name for word in ["pack","coffret","box","sac","offert","offerte","0fferte","0ffert"]):
                continue
            
            product = {
            #"index": index,
            #"category": "solar",
            #"category": "foodSup",
            "category": "faceGel",
            "product_name": None,
            "product_brand": None,
            "price": None,
            "promo": 0,
            "url": None,
            "ingredients":None,
            "description":None,
            "scraping_date": scraping_date
            }
            product["url"] = link.get_attribute('href')
            product["product_name"] = link.text.strip()

            brand = product_element.find_element(By.CSS_SELECTOR, 'div.stats > span > span:nth-child(2) > a')
            product["product_brand"] = brand.text.strip()

            promo_prices = product_element.find_elements(By.CSS_SELECTOR, 'div.caption > div.price > div > span.price-new')

            if promo_prices:
                product["promo"] = 1
                product["price"] = promo_prices[0].text.strip()
            else:
                regular_prices = product_element.find_elements(By.CSS_SELECTOR, 'div.caption > div.price > div > span')
                product["price"] = regular_prices[0].text.strip()
            product_data.append(product)
            

        except :
            print("error")
    
    return product_data

# Usage
#products = getFirstElements()
#print(f"Found {len(products)} products")
#for p in products:
    #print(p)


In [132]:
def getSecondElements(products):
    for product in products:
        try:
            chrome_driver.get(product["url"])
            time.sleep(3)
            desc_element = chrome_driver.find_element(By.CSS_SELECTOR, '#content > div.product-blocks.blocks-default > div')#'div.product-description')
            product["description"] = desc_element.text.strip()
            #print(product["description"])


        except Exception as e:
            #print(f"Error with product {index}: {e}")
            print("erreur")
    return(products)


    

In [135]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--headless")
chrome_driver = webdriver.Chrome(options=chrome_options)
page_url_solar ='https://www.parashop.tn/solaire?fc=345'
page_url_alim ='https://www.parashop.tn/nos-complements-alimentaires?fc=149'
page_url_gel='https://www.parashop.tn/visage/nettoyant-demaquillant/gel-lavant'

In [153]:
def execute_url(page_url):
    chrome_driver.get(page_url)
    #chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.8);")
    #collectProducts()  
    products_parashop = getFirstElements()
    print(f"Found {len(products_parashop)} products")
    getSecondElements(products_parashop)
    return(products_parashop)
    

In [140]:
products_parashop_solar=execute_url(page_url_solar)

error
Found 231 products
erreur
erreur
erreur
erreur
erreur
erreur


In [147]:
products_parashop_alim=execute_url(page_url_alim)

Reached the bottom!
/html/body/div[4]/div[2]/div/div/div/div[2]/div[49]/a
49
/html/body/div[4]/div[2]/div/div/div/div[2]/div[49]/a
49 FINAL
Found 44 products


In [154]:
products_parashop_gel=execute_url(page_url_gel)

Found 12 products


In [155]:
products_parashop=products_parashop_solar+products_parashop_alim+products_parashop_gel

In [156]:
with open('data_parashop.json', 'w', encoding='utf-8') as json_file:
                json.dump(products_parashop, json_file,
                          ensure_ascii=False)

# paralabel.tn

In [111]:
def collectProducts():
    while True:
        try:
            path='/html/body/main/section/div[2]/div/div[2]/section/section/div[3]/div/div[2]/a'
            click_plus= WebDriverWait(chrome_driver,5).until(EC.element_to_be_clickable((By.XPATH,path))).click()
            #print('clicked')
            chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.8);")
            time.sleep(2)
        except:
            break


    

In [127]:
def getFirstElements():
    products = chrome_driver.find_elements(By.CSS_SELECTOR, 'div.row.product_content.grid.row > div')
    product_data = []
    
    # Date as simple string
    scraping_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    #for index, product_element in enumerate(products, 1):
    for product_element in products:
        try:
            link = product_element.find_element(By.CSS_SELECTOR, 'div.inner_desc > p > a')
            name= link.text.strip().lower()
            if any(word in name for word in ["pack","coffret","box","sac","offert","offerte","0fferte","0ffert"]):
                continue
            product = {
            #"index": index,
            #"category": "solar",
            #"category": "foodSup",
            "category": "faceGel",
            "product_name": None,
            "product_brand": None,
            "price": None,
            "promo": 0,
            "url": None,
            "ingredients":None,
            "description":None,
            "scraping_date": scraping_date
            }
            product["url"] = link.get_attribute('href')
            product["product_name"] = link.text.strip()

            brand = product_element.find_element(By.CSS_SELECTOR, 'div.inner_desc > div.manufacturer > a')
            product["product_brand"] = brand.text.strip()


            regular_prices = product_element.find_elements(By.CSS_SELECTOR, ' div.product_desc > div.inner_desc > div.product-price-and-shipping > span')
            product["price"] = regular_prices[0].text.strip()
            product_data.append(product)
            
        except :
            print("error")
    
    return product_data

# Usage
#products_paralabel_solar = getFirstElements()
#print(f"Found {len(products_paralabel)} products")
#for p in products_paralabel:
    #print(p)


In [113]:
def getSecondElements(products):
    for product in products:
        try:
            chrome_driver.get(product["url"])
            time.sleep(3)
            desc_element = chrome_driver.find_element(By.CSS_SELECTOR, '#main > div > div.col-md-12.description-block-costumization > section > div')#'div.product-description')
            product["description"] = desc_element.text.strip()
            #print(product["description"])


        except Exception as e:
            print("erreur")
    return(products)

        

    

## Main

In [118]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--headless")
chrome_driver = webdriver.Chrome(options=chrome_options)
page_url_solar ='https://www.paralabel.tn/13-ecran-solaire-'
page_url_alim ='https://www.paralabel.tn/16-complements-alimentaires-'
page_url_gel='https://www.paralabel.tn/123-nettoyage'

In [119]:
def execute_url(page_url):
    chrome_driver.get(page_url)
    chrome_driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.8);")
    collectProducts()  
    products_paralabel = getFirstElements()
    print(f"Found {len(products_paralabel)} products")
    getSecondElements(products_paralabel)
    return(products_paralabel)
    

In [120]:
products_paralabel_solar=execute_url(page_url_solar)

Found 207 products
erreur
erreur
erreur
erreur
erreur


In [126]:
products_paralabel_alim=execute_url(page_url_alim)

Found 178 products
erreur
erreur
erreur
erreur
erreur
erreur


In [128]:
products_paralabel_gel=execute_url(page_url_gel)

Found 38 products


In [129]:
products_paralabel=products_paralabel_solar+products_paralabel_alim+products_paralabel_gel

In [130]:
with open('data_paralabel.json', 'w', encoding='utf-8') as json_file:
                json.dump(products_paralabel, json_file,
                          ensure_ascii=False)

# Save json

In [157]:
products_parasave=products_paralabel+products_parashop

In [158]:
with open('data_parasave.json', 'w', encoding='utf-8') as json_file:
                json.dump(products_parasave, json_file,
                          ensure_ascii=False)